In [1]:
# !wget https://huggingface.co/datasets/mesolitica/malaysian-benchmark/resolve/main/iium-coffession-en.json

In [14]:
from tqdm import tqdm
import requests
import os
import json

In [15]:
with open('iium-coffession-en.json') as fopen:
    d = json.load(fopen)
    
len(d)

1000

In [4]:
!mkdir small-iium-en

In [5]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-small-malaysian-translation-v2')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/nanot5-small-malaysian-translation-v2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
all_special_ids = [0, 1, 2]

In [8]:
l = d[0]['from']

In [9]:
input_ids = tokenizer.encode(f'terjemah ke Inggeris: {l}{tokenizer.eos_token}', return_tensors = 'pt')
outputs = model.generate(input_ids, max_length = 1024, num_beams=5, early_stopping=True)
outputs = [i for i in outputs[0] if i not in all_special_ids]
print(tokenizer.decode(outputs, spaces_between_special_tokens = False).strip())

The problem of acne will worsen if using alkaline/high pH facial cleansers, and when using low pH facial cleansers, acne will decrease and the skin will improve (If you are diligent, you can read the study Korting et al, the title 'The Influence of the Regular Use of a Soap or an Acidic Syndet Bar on Pre-Acne', not sure if you can find it on Google, but in the blog I read it, there is a direct link to the pdf file study).


In [12]:
for i in tqdm(range(len(d))):
    filename = os.path.join('small-iium-en', f'{i}.json')
    
    if os.path.exists(filename):
        continue
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    l = d[i]['from']
    
    input_ids = tokenizer.encode(f'terjemah ke Inggeris: {l}{tokenizer.eos_token}', return_tensors = 'pt')
    outputs = model.generate(input_ids, max_length = 1024, num_beams=5, early_stopping=True)
    outputs = [i for i in outputs[0] if i not in all_special_ids]
    r = tokenizer.decode(outputs, spaces_between_special_tokens = False).strip()

    with open(filename, 'w') as fopen:
        json.dump({'text': l, 'r': r}, fopen)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [08:11<00:00,  2.04it/s]


In [13]:
from sacrebleu.metrics import BLEU, CHRF, TER

chrf = CHRF(word_order = 2)

In [23]:
predicted, actual = [], []
for i in range(len(d)):
    try:
        filename = os.path.join('small-iium-en', f'{i}.json')
        with open(filename) as fopen:
            d_ = json.load(fopen)
        predicted.append(d_['r'])
        actual.append(d[i]['to'])
    except:
        pass

In [25]:
score = chrf.corpus_score(predicted, [actual])
score

chrF2++ = 55.24